### This code works fine but the GUI library has a glitch that it kills kernel second time you run. 
### That's why we run it as a subprocess 

In [47]:
import numpy as np
import open3d as o3d
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import copy
from PIL import Image
from io import BytesIO
import time

sample_ply_data = o3d.data.DemoCropPointCloud()
tst_pointCloud = o3d.io.read_point_cloud(sample_ply_data.point_cloud_path)
#o3d.visualization.draw_geometries([tst_pointCloud])

def array_to_data(array):
    im = Image.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1,origin=[0, 0, 0])
    mesh_pc= mesh.sample_points_uniformly(number_of_points=1000000, use_triangle_normal=False)
    pointCloud+=mesh_pc
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.radians(RX), np.radians(RY), np.radians((RZ))))
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    #time.sleep(5)
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color


In [48]:
dat=[]  #to hold cluster's images

clouds = [tst_pointCloud,tst_pointCloud]

for current_cloud in clouds:
        color = fetch_cloud_image(current_cloud, RX=120, RY=0, RZ=220)  #different point clouds to be viewed
        dat.append(color)

In [105]:
def Cluster_selection_gui(dat):
    
    selected_PC = [] #will hold index values of selected point clouds.
    selected_motion = 0  #default auto

    file_list_column = [[sg.Text("Select all valid Clusters"),]]
    for i in range (len(dat)):
        #print(i)
        file_list_column.append( [sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i), image_source=dat[i], 
                                     image_size = (None, None), image_subsample = 5, )] )

    buttons_layout = [
        [sg.Radio(text = "Auto select motion Path",default = True, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_Auto',
                  auto_size_text = True, group_id=0, pad = ((5, 0), (0, 8)))],
        
        [sg.Radio(text = "Motion along X-axis",default = False, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_X',
                  auto_size_text = True, group_id=0 )],
        
        [sg.Radio(text = "Motion along Y-axis",default = False, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_Y',
                  auto_size_text = True, group_id=0 ),          
         sg.Button(button_text = "Cancel",enable_events = True, tooltip ='First Cluster will be auto-selected', 
                                     size = (10, 2),key = 'Cancel', image_source=None,
                                     pad = ((60, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,
         sg.Button(button_text = "OK",enable_events = True, disabled = True, tooltip ='Select atleast one cluster',
                                     size = (10, 2),key = 'OK', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]
                 ]
    
    
    
    image_viewer_column = [
        [sg.Text("Preview")],
        [sg.Text(size=(40, 1), key="-TOUT-")],
        [sg.Image(key="img",source=dat[0])], [sg.HSeparator(pad=(0,10))], [sg.Column(buttons_layout)]  ]


    layout = [ [ sg.Column(file_list_column,size = (None, None), 
                           scrollable=True, vertical_scroll_only=True,expand_y=True ), 
                sg.VSeparator(), 
                sg.Column(image_viewer_column), ] 
             ]

    window = sg.Window("Pointcloud Browser", layout)

    while True: # Run the Event Loop
        event, values = window.read()
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        if event == "Cancel" or event == sg.WIN_CLOSED:
            selected_PC = [0] #default
            selected_motion = 0 #default
            break

        elif event == "Rad_Auto":
            selected_motion = 0 #default
        elif event == "Rad_X":  #curve along Y-axis and motion along X axis
            selected_motion = 2 
        elif event == "Rad_Y":  #curve along X-axis and motion along Y axis
            selected_motion = 3 

        elif event_name == "but":
            try:
                window["img"].update(source=dat[event_idx])
            except:
                print("Error")
                pass
            #if event_idx == 0:

            if (event_idx in selected_PC) == True:
                available_index = selected_PC.index(event_idx)
                del_item = selected_PC.pop(available_index)
                window["but"+str(event_idx)].update(text = "", image_data=dat[event_idx], image_subsample = 5)
                selected_PC = sorted(selected_PC)
                if len(selected_PC)==0:
                    window['OK'].set_tooltip("Select atleast one cluster")
                    window["OK"].update(disabled = True)
                #print(selected_PC)
            else:
                selected_PC.append(event_idx)
                window["but"+str(event_idx)].update(text = "SELECTED",  image_subsample = 5)
                window['OK'].set_tooltip("You may Proceed!")
                window["OK"].update(disabled = False)               
                selected_PC = sorted(selected_PC)
                #print(selected_PC)

        elif event == "OK":

            #print("The current selection includes:",selected_PC)
            break

    window.close()

    return selected_PC,selected_motion


new_one = Cluster_selection_gui(dat)
new_one

([0, 1], 3)